<a href="https://colab.research.google.com/github/semo970921/Hello_Buddy/blob/main/print(%22Hello_Buddy%22).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
from google.colab import files
import random

uploaded = files.upload()
파일_이름 = list(uploaded.keys())[0]
df = pd.read_excel(파일_이름, sheet_name='Sheet1')

# 닉네임과 관심분야가 모두 비어 있지 않은 행만 필터링
df = df.dropna(subset=['닉네임/이름', '관심분야'])


Saving 통합 문서1.xlsx to 통합 문서1 (1).xlsx


In [16]:
# 닉네임과 관심분야를 리스트로
닉네임_관심분야 = [(row['닉네임/이름'], str(row['관심분야']).split(', ') if pd.notna(row['관심분야']) else []) for index, row in df.iterrows()]

# 두 사람의 공통 관심사 개수를 계산하는 함수
def 공통_관심사_개수(관심사1, 관심사2):
    return len(set(관심사1) & set(관심사2))


In [17]:
매칭_결과 = []
사용된_닉네임 = set()

# 찐친들 예외 처리
제외_짝 = [
    ('민트/박하은', '제이/박민음'), ('제이/박민음', '민트/박하은'),
    ('올리브', '양양'), ('양양', '올리브'),
    ('민트', '양양'), ('양양', '민트'),
    ('찬희', '대니'), ('대니', '찬희'),
    ('덕규', '네오'), ('네오', '덕규'),
    ('리브', '제이'), ('제이', '리브'),
    ('리브', '민'), ('민', '리브'),
    ('제이', '민'), ('민', '제이'),
    ('리브', '차차'), ('차차', '리브'),
    ('민트', '차차'), ('차차', '민트'),
    ('리브', '콩성'), ('콩성', '리브'),
    ('리브', '웨인'), ('웨인', '리브'),
    ('콩성', '웨인'), ('웨인', '콩성'),
    ('콩성', '제로'), ('제로', '콩성'),
    ('웨인', '제로'), ('제로', '웨인'),
    ('콩성', '가프'), ('가프', '콩성'),
    ('이현', '도하'), ('도하', '이현'),
    ('도하', '네모'), ('네모', '도하'),
    ('네모', '나나'), ('나나', '네모')
]

In [22]:
for i, (닉네임1, 관심사1) in enumerate(닉네임_관심분야):
    if 닉네임1 in 사용된_닉네임:
        continue  # 이미 매칭된 닉네임

    후보_매칭 = []

    for j, (닉네임2, 관심사2) in enumerate(닉네임_관심분야):
        # 제외된 페어 매칭 방지
        if (닉네임1, 닉네임2) in 제외_짝 or (닉네임2, 닉네임1) in 제외_짝:
            continue

        if i != j and 닉네임2 not in 사용된_닉네임:
            공통_개수 = 공통_관심사_개수(관심사1, 관심사2)
            if 공통_개수 > 0:
                후보_매칭.append((닉네임2, 공통_개수))

    # 공통 관심사가 많은 순으로 정렬
    if 후보_매칭:
        후보_매칭.sort(key=lambda x: x[1], reverse=True)

        최대_공통_개수 = 후보_매칭[0][1]
        가장_좋은_후보들 = [매칭 for 매칭 in 후보_매칭 if 매칭[1] == 최대_공통_개수]
        선택된_매칭 = random.choice(가장_좋은_후보들)[0] # 동점일 경우 랜덤 선택

        매칭_결과.append((닉네임1, 선택된_매칭, 최대_공통_개수))
        사용된_닉네임.add(닉네임1)
        사용된_닉네임.add(선택된_매칭)


In [23]:
# 공통 관심사로 매칭되지 않은 멤버들을 위한 매칭
남은_멤버 = [닉네임 for 닉네임, _ in 닉네임_관심분야 if 닉네임 not in 사용된_닉네임]

random.shuffle(남은_멤버)
for i in range(0, len(남은_멤버), 2):
    if i+1 < len(남은_멤버):
        매칭_결과.append((남은_멤버[i], 남은_멤버[i+1], "랜덤 매칭"))
    elif i == len(남은_멤버) - 1:   # 남은 한 명은 3인 그룹으로 묶음
        마지막_멤버 = 남은_멤버[i]
        가장_좋은_그룹 = None
        최대_공통_그룹_개수 = 0

        # 이미 매칭된 그룹 중에서 공통 관심사가 가장 많은 그룹을 찾음
        for idx, (닉네임2, 닉네임3, _) in enumerate(매칭_결과):
            관심사2 = next((관심사 for 이름, 관심사 in 닉네임_관심분야 if 이름 == 닉네임2), [])
            관심사3 = next((관심사 for 이름, 관심사 in 닉네임_관심분야 if 이름 == 닉네임3), [])
            공통_그룹_개수 = 공통_관심사_개수(닉네임_관심분야[next((i for i, x in enumerate(닉네임_관심분야) if x[0] == 마지막_멤버))][1], 관심사2) + 공통_관심사_개수(닉네임_관심분야[next((i for i, x in enumerate(닉네임_관심분야) if x[0] == 마지막_멤버))][1], 관심사3)

            if 공통_그룹_개수 > 최대_공통_그룹_개수:
                최대_공통_그룹_개수 = 공통_그룹_개수
                가장_좋은_그룹 = idx

        # 공통 관심사가 많은 그룹에 마지막 멤버 추가하고, 각 멤버 간의 공통 관심사 개수 표시
        if 가장_좋은_그룹 is not None:
            닉네임2, 닉네임3, 기존_공통_개수 = 매칭_결과[가장_좋은_그룹]
            관심사2 = next((관심사 for 이름, 관심사 in 닉네임_관심분야 if 이름 == 닉네임2), [])
            관심사3 = next((관심사 for 이름, 관심사 in 닉네임_관심분야 if 이름 == 닉네임3), [])
            마지막_멤버_관심사 = 닉네임_관심분야[next((i for i, x in enumerate(닉네임_관심분야) if x[0] == 마지막_멤버))][1]

            공통_개수_멤버1_추가 = 공통_관심사_개수(관심사2, 마지막_멤버_관심사)
            공통_개수_멤버2_추가 = 공통_관심사_개수(관심사3, 마지막_멤버_관심사)

            매칭_결과[가장_좋은_그룹] = (닉네임2, 닉네임3, f'{마지막_멤버} (멤버3추가/멤버1&2의 공통:{기존_공통_개수}개,1&3 공통:{공통_개수_멤버1_추가}개,2&3 공통:{공통_개수_멤버2_추가}개)')



In [ ]:
# 내림차순으로 정렬
매칭_결과.sort(key=lambda x: (x[2] if isinstance(x[2], int) else 0), reverse=True)


매칭_결과_df = pd.DataFrame(매칭_결과, columns=["닉네임1", "닉네임2", "공통 관심사 수"])
print(매칭_결과_df)
